In [1]:
## Assignment 1 Intelligent Systems 

In [2]:
import pygad
import pandas as pd
import numpy as np

# Load the JSON file into a pandas DataFrame


In [3]:
#  Converting the preference data into an appropriate format 


df = pd.read_json('easy_dataset_1.json')
#First drop unnecessary data and the them into variables 
num_papers = df.loc[0,"num_papers"]
df = df.drop(columns="num_papers")
num_reviewers = df.loc[0,"num_reviewers"]
df = df.drop(columns="num_reviewers")
reviewer_capacity = df.loc[0,"reviewer_capacity"]
df = df.drop(columns ="reviewer_capacity")
min_reviews_per_paper = df.loc[0,"min_reviews_per_paper"]
df = df.drop(columns="min_reviews_per_paper")
max_reviews_per_paper = df.loc[0,"max_reviews_per_paper"]
df = df.drop(columns="max_reviews_per_paper")
df = df.drop(columns="dataset_id")
print(df)




preferences_matrix = np.array(df["preferences"].tolist())
friendships_matrix = np.array(df["friendships"].tolist())
authorship_matrix = np.array(df["authorship"].tolist())
print(num_papers)

       preferences      friendships       authorship
0  [3, 2, 1, 5, 3]  [0, 0, 0, 0, 0]  [0, 0, 0, 0, 0]
1  [5, 2, 4, 2, 2]  [0, 0, 1, 0, 0]  [0, 1, 0, 0, 0]
2  [4, 3, 1, 2, 4]  [0, 1, 0, 0, 0]  [0, 0, 0, 0, 0]
3  [4, 2, 2, 1, 1]  [0, 0, 0, 0, 0]  [1, 1, 0, 0, 1]
4  [4, 3, 4, 1, 5]  [0, 0, 0, 0, 0]  [0, 0, 1, 1, 0]
5


In [4]:
# Example assign matrix for given dataset 
assign_matrixnp = np.zeros((5,5), dtype= int)
assign_matrixnp[0,2] = 1
assign_matrixnp[0,3] = 1
assign_matrixnp[0,4] = 1
assign_matrixnp[0,1] = 1
assign_matrixnp[1,2] = 1
assign_matrixnp[2,4] = 1
assign_matrixnp[3,0] = 1
assign_matrixnp[4,3] = 1
print(assign_matrixnp)

[[0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 1 0]]


In [5]:
#compute fitness value in absolute or relative 
def f(assign_matrix, preferences_matrix, friendships_matrix, authorship_matrix): 
    
    fit_val = 0 
   
    # penalty scores table/distribution
    preference_points = np.array([[0,1,2,3,4,5],[3,2,1,0,-1,-2]])
    penalty_reviewer_cap = 5
    penalty_authorship = 9
    penalty_friendship = 7
    penalty_low_reviewcount= 7
    penalty_high_reviewcount = 3 
    

    # iterate over reviewers and papers
    for i in range(len(assign_matrix[:,0])): 
        for j in range(len(assign_matrix[0,:])): 
            # add the preference penalty for the assigned paper
            if assign_matrix[i,j] == 1: 
                index = preferences_matrix[i,j]
                fit_val += preference_points[1, index]
                # get other y of reviewed paper j 
                for y in range(len(assign_matrix[0,:])): 
                    if authorship_matrix[y,j] == 1:
                        author = y
                #if reviewer and author are friends assign friendconstraint penalty 
                if friendships_matrix[author,i] == 1: 
                    fit_val+= penalty_friendship
                    #print("friendship penalty")
            # if assigned reviewer is also author add authorconstraint penalty
            if assign_matrix[i,j] == 1 and authorship_matrix[i,j] ==1:
                fit_val+= penalty_authorship     
                #print("authorship penalty")       
        
    # penalties for reviewer constraints violations              
    for t in range(len(assign_matrix[:,0])): 
        x = np.sum(assign_matrix[i,t])
        if x > reviewer_capacity:
            fit_val += penalty_reviewer_cap
            #print("penalty reviewer cap")
    
    # penalties with the amount of reviews needed is not met worse for less reviews than for 2 much reviews
    for z in range(len(assign_matrix[0,:])):
        x = np.sum(assign_matrix[:,z])
        if x < min_reviews_per_paper:
            fit_val += (min_reviews_per_paper - x)*penalty_low_reviewcount
            #print("penalty for to less reviews for paper")
        if x > max_reviews_per_paper:
            fit_val += (x - max_reviews_per_paper)*penalty_high_reviewcount
    # problem that low fitness function is better -> assign minus      
    fit_val = -1*fit_val    
    return fit_val 

print(f(assign_matrixnp, preferences_matrix, friendships_matrix, authorship_matrix ))



-67


In [6]:
#Utilize PyGad 

#pyGad takes as a solution max 2D Arrays to flatten reviewer x paper array x num solutions in num solutions x values
def fitness_function(ga_instance, solution, solution_idx):
    return f(solution.reshape(num_reviewers,num_papers), preferences_matrix, friendships_matrix, authorship_matrix)   

#create initial position with random values  
def custom_initialpop(num_solutions, num_reviewers, num_papers):
    pop = np.random.randint(0,2, size=(num_solutions, num_reviewers, num_papers))
    return pop.reshape((num_solutions,-1))

# helper function to check iff an assignment violates author or friendship constraint
def is_author_orfriend (reviewer, paper): 
    is_relation = False
    if (authorship_matrix[reviewer, paper] == 1):
        is_relation = True
    authors = np.where(authorship_matrix[:,paper] ==1)[0]
    #print("author:",authors)

    for author in authors:
        if friendships_matrix[author, reviewer] == 1:
            is_relation = True
    return is_relation 



In [7]:
## custom crossover functions that return offspring which fullfill the constraints 

def custom_crossover(parents, offspring_size, ga_instance):
    offspring = []

    ## Do initial crossover inherit each column/ paper assigned to authors for either parent 1 or 2 

    for i in range(parents.shape[0]):
        parent1 = parents[i % parents.shape[0], :].copy()
        parent2 = parents[(i+1) % parents.shape[0], :].copy()

        # SWAP Papers (Columns) the Assignmentmatrix

        # reshape for easier computation 
        parent1 = parent1.reshape(num_reviewers,num_papers)
        parent2 = parent2.reshape(num_reviewers,num_papers)
        
        # equally the amount of columns given to the child by each parent
        
        hereditary_parent1 = num_papers // 2 #whole number divide operator 
        hereditary_parent2 = num_papers - hereditary_parent1
        # calcs a 1D-Array with indices which columns(papers) are given to the child afterwards
        indices_parent1 = np.random.choice(range(num_papers), size=hereditary_parent1, replace=False)
        indices_parent2 = [i for i in range(num_papers) if i not in indices_parent1]

        # computes child out of the randomized indices
        new_child = np.zeros_like(parent1)
        new_child[:, indices_parent1] = parent1[:, indices_parent1]
        new_child[:, indices_parent2] = parent2[:, indices_parent2]
        
        offspring.append(new_child)

    offspring = np.array(offspring)
    

    ## Check if reviewer capacity is violated and that remove papers 

    # for every child
    for i in range(offspring.shape[0]):
        # for every row / reviewer 
        for j in range(offspring.shape[1]): 
            if np.sum(offspring[i,j,:]) > reviewer_capacity: 
                # Number of assignments that need to be removed from this reviewer
                to_remove = np.sum(offspring[i,j,:]) - reviewer_capacity
                # perform the remove operations as many as to remove 
                for y in range(to_remove): 
                    #get assigned indexes and choose a random idx from that and remove the assignment
                    idx_assigned = np.where(offspring[i,j,:] == 1)[0]
                    idx_delete = np.random.choice(idx_assigned, size=1, replace=False)
                    offspring[i,j,idx_delete] = 0
                    
    ## add reviewer assignment to fullfill review_requirements just add assignment if no other constraints are violated

    # iterate over every child in offspring
    for i in range(offspring.shape[0]):
        #iterate over every col / paper 
        for j in range(offspring.shape[2]):
            
            # do corrections if review requierements now dont fit anymore
            if np.sum(offspring[i,:,j]) < min_reviews_per_paper: 
                
                # available reviewers dont have the paper assigned yet and are under their reviewer_capacity
                available_reviewers = np.where((offspring[i,:,j] == 0) & (np.sum(offspring[i,:,:], axis=1) < reviewer_capacity))[0]
                
                for reviewer in available_reviewers: 
                    #check whether potential reviewers is friends or author 
                    if (is_author_orfriend(reviewer, j) == False): 
                        offspring[i,reviewer,j] = 1
                        break

                    
    return offspring.reshape(offspring.shape[0], offspring.shape[1]*offspring.shape[2])                    


In [8]:
## custom_mutation function 

def custom_mutation(offspring, ga_instance):
    #reshape to num_reviewers, num_papers again
    offspring = offspring.reshape(offspring.shape[0], num_reviewers, num_papers)
    num_mutations = (np.round(num_reviewers*num_papers*0.05)).astype(int)
    #print(range(num_mutations))
    
    # do mutation for every child in the offspring
    for i in range(offspring.shape[0]):
            
        #print(offspring[i])
        
        for j in range(num_mutations):

            ## Add Assignment 

            idx_assign = -1
            random_col = -1

            cols = np.where(np.sum(offspring[i, :, :], axis=0) < min_reviews_per_paper)[0]

            # exit mutation for this offspring when no potential colums are there
            if len(cols) == 0: 
                #print("no cols possible")
                break
            #print("cols possible:", cols)

            # choose random possible column and then random reviewer 
            while len(cols) > 0:

                random_col = np.random.choice(cols, size=1, replace=False)

                idxs_notassigned = np.where(offspring[i,:,random_col] == 0)[0]

                is_valid = False 
                # if there are possible reviewer and if they are not author or friend assign the paper
                # otherwise next possible reviewer
                while len(idxs_notassigned) > 0: 
                    idx_assign = np.random.choice(idxs_notassigned, size=1)[0]

                    if not is_author_orfriend(idx_assign, random_col): 
                        offspring[i,idx_assign, random_col] = 1
                        is_valid = True
                        #print("Mutation added at:", idx_assign, random_col)
                        #print(offspring[i])
                        break #dont look in this column anymore
                    else:
                        idxs_notassigned = np.delete(idxs_notassigned, np.where(idxs_notassigned == idx_assign))
                   
                
                if is_valid == True: 
                    break #dont look in another column anymore
                else: 
                    cols = np.delete(cols, np.where(cols = random_col))

            ## remove Assignment 

            # check for reviewer cap violation through last assignment
            if np.sum(offspring[i, idx_assign, :]) > reviewer_capacity:
                reviewer_assignments = np.where(offspring[i, idx_assign, :] == 1)[0]
                reviewer_assignments = np.delete(reviewer_assignments, np.where(reviewer_assignments == random_col))

                # check all columns of assigned papers if they have surplus reviews if yes remove that review
                is_reset = False
                for paper in reviewer_assignments:
                    if np.sum(offspring[i, :, paper]) > min_reviews_per_paper:
                        offspring[i, idx_assign, paper] = 0
                        #print("removed assignment cuz review cap violations surplus", idx_assign, paper)
                        #print(offspring[i])
                        is_reset = True
                        break
                # if no surplus review just remove random review        
                if not is_reset:
                    # Choose a random paper to unassign
                    random_paper = np.random.choice(reviewer_assignments, size=1)[0]
                    offspring[i, idx_assign, random_paper] = 0
                    #print("removed assignment cuz review cap vio random:", idx_assign, random_paper)
                    #print(offspring[i])
            # If no reviewer cap is violated, try removing an assignment from a paper with excess reviewers
            else:
                papers_with_excess = np.where(np.sum(offspring[i, :, :], axis=0) > min_reviews_per_paper)[0]
                papers_with_excess = np.delete(papers_with_excess, np.where(papers_with_excess == random_col))  # Exclude the just-assigned column

                if len(papers_with_excess) > 0:
                    random_paper = np.random.choice(papers_with_excess, size=1)[0]
                    assigned_reviewers = np.where(offspring[i, :, random_paper] == 1)[0]
                    if len(assigned_reviewers) > 0:
                        random_reviewer = np.random.choice(assigned_reviewers, size=1)[0]
                        offspring[i, random_reviewer, random_paper] = 0
                        #print("removed assignement:", random_reviewer, random_paper)
                        #print(offspring[i])
                else:
                    # If no paper with excess exists, remove any random valid assignment
                    all_assigned = np.array(np.where(offspring[i, :, :] == 1)).T
                    valid_points = [point for point in all_assigned if point[0] != idx_assign or point[1] != random_col]

                    if len(valid_points) > 0:
                        random_point = valid_points[np.random.choice(len(valid_points))]
                        offspring[i, random_point[0], random_point[1]] = 0
                        #print("removed assignemnt random valid assignment:", random_point[0], random_point[1])
                        #print(offspring[i])
    return offspring.reshape(offspring.shape[0], num_reviewers*num_papers)

In [16]:
##alternative version that is easier 

def custom_mutation_easy(offspring, ga_instance):
    #reshape to num_reviewers, num_papers again
    offspring = offspring.reshape(offspring.shape[0], num_reviewers, num_papers)
    num_mutations = (np.round(num_reviewers*num_papers*0.05)).astype(int)
    #print(range(num_mutations))
    
    # do mutation for every child in the offspring
    for i in range(offspring.shape[0]):
            
        #print(offspring[i])
        
        for j in range(num_mutations):

            ## Add Assignment 

            idx_assign = -1
            random_col = -1

            cols = np.arange(0,num_papers)
                       
                         
            # choose random possible column and then random reviewer 
            while len(cols) > 0:

                random_col = np.random.choice(range(num_papers), size=1, replace=False)
                #print("randomcol:", random_col)
                idxs_notassigned = np.where(offspring[i,:,random_col] == 0)[1]
                #print("not assigned idxs",idxs_notassigned)
                is_valid = False 
                # if there are possible reviewer and if they are not author or friend assign the paper
                # otherwise next possible reviewer
                while len(idxs_notassigned) > 0: 
                    idx_assign = np.random.choice(idxs_notassigned, size=1)[0]

                    if not is_author_orfriend(idx_assign, random_col): 
                        offspring[i,idx_assign, random_col] = 1
                        #print("Mutation Added", idx_assign, random_col)
                        is_valid = True
                        #print("Mutation added at:", idx_assign, random_col)
                        #print(offspring[i])
                        break #dont look in this column anymore
                    else:
                        idxs_notassigned = np.delete(idxs_notassigned, np.where(idxs_notassigned == idx_assign))
                              
                if is_valid == True: 
                    break #dont look in another column anymore
                else: 
                    cols = np.delete(cols, np.where(cols == random_col)[0])

            ## remove Assignment 

            # check for reviewer cap violation through last assignment
            if np.sum(offspring[i, idx_assign, :]) > reviewer_capacity:
                reviewer_assignments = np.where(offspring[i, idx_assign, :] == 1)[0]
                reviewer_assignments = np.delete(reviewer_assignments, np.where(reviewer_assignments == random_col))

                random_paper = np.random.choice(reviewer_assignments, size=1)[0]
                offspring[i, idx_assign, random_paper] = 0
                #print("removed assignment cuz review cap vio random:", idx_assign, random_paper)
                    #print(offspring[i])

            # if no reviewer cap violation remove any assignment which is not the just added Assignment   
            else:                
                all_assigned = np.array(np.where(offspring[i, :, :] == 1)).T
                valid_points = [point for point in all_assigned if point[0] != idx_assign or point[1] != random_col]

                if len(valid_points) > 0:
                    random_point = valid_points[np.random.choice(len(valid_points))]
                    offspring[i, random_point[0], random_point[1]] = 0
                    #print("removed assignemnt random valid assignment:", random_point[0], random_point[1])
                #print(offspring[i])
    return offspring.reshape(offspring.shape[0], num_reviewers*num_papers)

 

In [17]:
#initiate ga_instance
ga_instance = pygad.GA(fitness_func=fitness_function,
                       parent_selection_type = "tournament",
                       crossover_type= custom_crossover,
                       mutation_type = custom_mutation,
                       num_generations=10,      # number of generations
                       num_parents_mating=2,    #                   
                       sol_per_pop=10,          # solutions per pop
                       initial_population = custom_initialpop(10,num_reviewers,num_papers), #initialpopulation
                       gene_type=int,                          
                       num_genes= num_reviewers*num_papers,
                       gene_space = [0,1])       
                       
                                                     
#ga_instance.run()
aa = custom_initialpop(10,5,5)
#print(aa.reshape(10,num_reviewers,num_papers))

#print(ga_instance.cal_pop_fitness())

### Example code for parents to work with 
num_parents = 4  
num_genes = 25   
# this is just random not constraints respected 
parents = np.random.randint(0, 2, size=(num_parents, num_genes))

    
x = custom_crossover(parents, (4,25), ga_instance)
print("PostCross:",x[0].reshape(5,5))
z = custom_mutation_easy(x, ga_instance)

print("PostMut",z[0].reshape(5,5))


p1 = f(parents[0].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
p2 = f(parents[1].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)  
p3 = f(parents[2].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
p4 = f(parents[3].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)



x0 = f(x[0].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
x1 = f(x[1].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
x2 = f(x[2].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
x3 = f(x[3].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
#print(x[0].reshape(5,5))
##print("initial mean:",(p1+p2+p3+p4)/2)
#print("crossover mean:",(x1+x2+x3+x0)/2)


PostCross: [[1 0 1 1 0]
 [0 1 1 0 1]
 [1 1 0 1 0]
 [1 1 1 0 0]
 [0 1 1 0 1]]
PostMut [[1 0 1 1 0]
 [0 1 1 0 1]
 [1 0 1 1 0]
 [1 1 1 0 0]
 [0 1 1 0 1]]
